In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from line_profiler import LineProfiler
from sklearn.metrics import adjusted_rand_score

%load_ext Cython

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,5)

In [3]:
def profile_print(func_to_call, *args):
    profiler = LineProfiler()
    profiler.add_function(func_to_call)
    profiler.runcall(func_to_call, *args)
    profiler.print_stats()

<h2>Подготовка данных:</h2>

In [4]:
df_sns = pd.read_csv('snsdata.csv', sep=',')
df_sns.head()

,gradyear,gender,age,friends,basketball,football,soccer,softball,volleyball,swimming,...,blonde,mall,shopping,clothes,hollister,abercrombie,die,death,drunk,drugs
0,2006,M,18.982,7,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2006,F,18.801,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,2006,M,18.335,69,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,2006,F,18.875,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2006,NaN,18.995,10,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,1,1


In [5]:
#отделяем ненужные признаки
trash, data = df_sns.iloc[:, :4], df_sns.iloc[:, 4:]
df = pd.DataFrame(data)

#нормализация
df_norm = (df - df.mean()).div(df.std())
df_norm.head()

,basketball,football,soccer,softball,volleyball,swimming,cheerleading,baseball,tennis,sports,...,blonde,mall,shopping,clothes,hollister,abercrombie,die,death,drunk,drugs
0,-0.332212,-0.357691,-0.24287,-0.217924,-0.223666,-0.259966,-0.207324,-0.201127,-0.168936,-0.297118,...,-0.050936,-0.369909,-0.487306,-0.314193,-0.201473,-0.183029,-0.294788,-0.261526,-0.220399,-0.174905
1,-0.332212,1.060031,-0.24287,-0.217924,-0.223666,-0.259966,-0.207324,-0.201127,-0.168936,-0.297118,...,-0.050936,1.067374,-0.487306,-0.314193,-0.201473,-0.183029,-0.294788,-0.261526,-0.220399,-0.174905
2,-0.332212,1.060031,-0.24287,-0.217924,-0.223666,-0.259966,-0.207324,-0.201127,-0.168936,-0.297118,...,-0.050936,-0.369909,-0.487306,-0.314193,-0.201473,-0.183029,-0.294788,2.027874,-0.220399,-0.174905
3,-0.332212,-0.357691,-0.24287,-0.217924,-0.223666,-0.259966,-0.207324,-0.201127,-0.168936,-0.297118,...,-0.050936,-0.369909,-0.487306,-0.314193,-0.201473,-0.183029,-0.294788,-0.261526,-0.220399,-0.174905
4,-0.332212,-0.357691,-0.24287,-0.217924,-0.223666,-0.259966,-0.207324,-0.201127,-0.168936,-0.297118,...,-0.050936,-0.369909,2.273635,-0.314193,-0.201473,-0.183029,-0.294788,-0.261526,2.285084,2.719271


In [6]:
X = np.array(df_norm)

<h2>Алгоритм с использованием циклов:</h2>

In [6]:
from sklearn.base import ClusterMixin, BaseEstimator
from sklearn.metrics.pairwise import pairwise_distances, distance_metrics
import copy

class Kmeans(BaseEstimator, ClusterMixin): 
    
    def __init__(self, k=2, metric='euclidean', max_iter=1000, random_state=None):
        """
        Инициализация метода
        :k - количество кластеров
        :metric - функция расстояния между объектами
        :max_iter - максиальное количество итераций
        :random_state - seed для инициализации генератора случайных чисел
        """
        
        self.k = k
        self.random_state = random_state
        self.metric = metric
        self.max_iter = max_iter

    def fit(self, X, y=None):
        """
        Процедура обучения k-means
        """
        try:
            X = np.array(X)
        except:
            raise TypeError('Need pd.DataFrame or np.array')
        
        # Инициализация генератора случайных чисел
        np.random.seed(self.random_state)
        
        # Массив с метками кластеров для каждого объекта из X
        self.labels = np.empty(X.shape[0]).astype(int)
        new_labels = np.empty(X.shape[0]).astype(int)
        
        # Массив с центроидами кластеров, заполняем случайными числами в соответствии с диапазоном входных данных
        self.centroids = np.empty((self.k, X.shape[1]))
        new_centroids = np.empty((self.k, X.shape[1]))
        for i in xrange(X.shape[1]):
            min = X[:,i].min()
            max = X[:,i].max()
            self.centroids[:, i] = min + np.random.random(self.k)*(max-min)
                
        for it in xrange(self.max_iter):
            #print it
            #заполняем new_labels - ближайший центроид для каждого объекта
            for pt in xrange(X.shape[0]):
                min_dist = np.inf
                for cen in xrange(self.k):
                    #d = pairwise_distances(X[pt], self.centroids[cen], metric=self.metric)[0][0]
                    d = distance_metrics()[self.metric]([self.centroids[cen]], [X[pt]])
                    #print d
                    if d < min_dist:
                        min_dist = d
                        new_labels[pt] = cen
            
            #считаем новые координаты центроидов
            num_pts_in_cluster = np.zeros(self.k).astype(int)
            for pt in xrange(X.shape[0]):
                new_centroids[new_labels[pt]] += X[pt]
                num_pts_in_cluster[new_labels[pt]] += 1
            for cen in xrange(self.k):
                if num_pts_in_cluster[cen] != 0:
                    new_centroids[cen] /= num_pts_in_cluster[cen]
                else:
                    new_centroids[cen] = self.centroids[cen]        
            
            #если среднее смещение центроида меньше eps, то заканчиваем
            #delta = float(np.trace(distance_metrics()[self.metric](self.centroids, new_centroids))/self.k)
            #eps = 0.001
            #if delta < eps:
            
            #если ни один объект не поменял свой кластер, то заканчиваем
            if np.array_equal(new_labels, self.labels):
                self.centroids = copy.deepcopy(new_centroids)
                self.labels = copy.deepcopy(new_labels)
                #print 'iterations = ', it
                break
            
            self.centroids = copy.deepcopy(new_centroids)
            self.labels = copy.deepcopy(new_labels)

        return self

    def predict(self, X, y=None):
        """
        Процедура предсказания кластера
        
        Возвращает метку ближайшего кластера для каждого объекта
        """
        try:
            X = np.array(X)
        except:
            raise TypeError('Need pd.DataFrame or np.array')
        
        return distance_metrics()[self.metric](self.centroids, X).argmin(axis=0)


Очень медленно, даже на 1000 точек:

In [7]:
%timeit -n 3 -r 3 Kmeans(k=9, random_state=322).fit(df_norm[:1000])

3 loops, best of 3: 3.08 s per loop


In [8]:
f = Kmeans(k=9, random_state=322).fit
profile_print(f, df_norm[:100])

Timer unit: 1e-06 s

Total time: 0.588246 s
File: <ipython-input-6-206fd6f176c5>
Function: fit at line 21

Line #      Hits         Time  Per Hit   % Time  Line Contents
    21                                               def fit(self, X, y=None):
    22                                                   """
    23                                                   Процедура обучения k-means
    24                                                   """
    25         1            3      3.0      0.0          try:
    26         1          128    128.0      0.0              X = np.array(X)
    27                                                   except:
    28                                                       raise TypeError('Need pd.DataFrame or np.array')
    29                                                   
    30                                                   # Инициализация генератора случайных чисел
    31         1           15     15.0      0.0          np.random.seed(s

Как и ожидалось, больше всего времени занимает вычисление расстояний в цикле между точками и центроидами - 96%.

<h2>Векторизованный алгоритм:</h2>

In [26]:
from sklearn.base import ClusterMixin, BaseEstimator
from numpy.linalg import norm
from sklearn.metrics.pairwise import distance_metrics
import copy

class Kmeans(BaseEstimator, ClusterMixin): 
    
    def __init__(self, k=2, metric='euclidean', max_iter=1000, random_state=None):
        """
        Инициализация метода
        :k - количество кластеров
        :metric - функция расстояния между объектами
        :max_iter - максиальное количество итераций
        :random_state - seed для инициализации генератора случайных чисел
        """
        
        self.k = k
        self.random_state = random_state
        self.metric = metric
        self.max_iter = max_iter

    def fit(self, X, y=None):
        """
        Процедура обучения k-means
        """
        try:
            X = np.array(X)
        except:
            raise TypeError('Need pd.DataFrame or np.array')
        
        # Инициализация генератора случайных чисел
        np.random.seed(self.random_state)
        
        # Массив с метками кластеров для каждого объекта из X
        self.labels = np.empty(X.shape[0], dtype=np.int64)
        new_labels = np.empty(X.shape[0], dtype=np.int64)
        
        # Массив с центроидами кластеров, заполняем случайными числами в соответствии с диапазоном входных данных
        self.centroids = np.empty((self.k, X.shape[1]), dtype=np.float64)
        new_centroids = np.empty((self.k, X.shape[1]), dtype=np.float64)
        for i in xrange(X.shape[1]):
            min = X[:,i].min()
            max = X[:,i].max()
            self.centroids[:, i] = min + np.random.random(self.k)*(max-min)
 

        for it in xrange(self.max_iter):
            #заполняем new_labels - ближайший центроид для каждого объекта
            new_labels = distance_metrics()[self.metric](self.centroids, X).argmin(axis=0)
                        
            #считаем новые координаты центроидов
            for cen in xrange(self.k):
                new_centroids[cen] = np.average(X[new_labels == cen], axis=0)
            
            #если нет ни одной точки в кластере, снова случайно выставляем этот центроид
            empty_clusters = np.argwhere(np.isnan(new_centroids).any(axis=1))
            new_centroids[empty_clusters] = self.centroids[empty_clusters]         
            
            #если среднее смещение центроида меньше eps, то заканчиваем
            #delta = float(np.trace(distance_metrics()[self.metric](self.centroids, new_centroids))/self.k)
            #eps = 0.001
            #if delta < eps:
            
            #если ни один объект не поменял свой кластер, то заканчиваем
            if np.array_equal(new_labels, self.labels):
                np.copyto(self.centroids, new_centroids)
                np.copyto(self.labels, new_labels)
                #print 'iterations = ', it
                break
            
            np.copyto(self.centroids, new_centroids)
            np.copyto(self.labels, new_labels)

        return self

    def predict(self, X, y=None):
        """
        Процедура предсказания кластера
        
        Возвращает метку ближайшего кластера для каждого объекта
        """
        try:
            X = np.array(X)
        except:
            raise TypeError('Need pd.DataFrame or np.array')
        
        return distance_metrics()[self.metric](self.centroids, X).argmin(axis=0)


На том же объеме данных стало быстрее более чем в 200 раз: 

In [11]:
%timeit Kmeans(k=9, random_state=322).fit(df_norm[:1000])

10 loops, best of 3: 17.2 ms per loop


На полном объеме данных тоже работает достаточно быстро:

In [12]:
%timeit -n 3 -r 3 Kmeans(k=9, random_state=322).fit(df_norm)

3 loops, best of 3: 551 ms per loop


Сохраним метки для точек, чтобы в дальнейшем сравнивать с ними метки, полученные от других реализаций - для проверки корректности.

In [27]:
true_labels = Kmeans(k=9, random_state=322).fit(X).labels

/home/ivan/anaconda2/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/home/ivan/anaconda2/lib/python2.7/site-packages/numpy/core/_methods.py:68: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


In [28]:
from collections import Counter
lbl = Kmeans(k=9, random_state=322).fit(X).labels
cnt = Counter(lbl)
print cnt

Counter({5: 22492, 4: 5284, 8: 1069, 1: 598, 6: 556, 0: 1})


In [15]:
f = Kmeans(k=9, random_state=322).fit
profile_print(f, df_norm)

Timer unit: 1e-06 s

Total time: 0.688757 s
File: <ipython-input-9-6518b715327d>
Function: fit at line 22

Line #      Hits         Time  Per Hit   % Time  Line Contents
    22                                               def fit(self, X, y=None):
    23                                                   """
    24                                                   Процедура обучения k-means
    25                                                   """
    26         1            3      3.0      0.0          try:
    27         1         2115   2115.0      0.3              X = np.array(X)
    28                                                   except:
    29                                                       raise TypeError('Need pd.DataFrame or np.array')
    30                                                   
    31                                                   # Инициализация генератора случайных чисел
    32         1           24     24.0      0.0          np.random.seed(s

Самые долгие процедуры - вычисление минимальных расстояний (48%), а потом новых центроидов (49%). Обе задачи векторизованы, написаны в одну строчку. Есть только один цикл по центроидам, но он небольшой, всего k итераций, и непонятно, как от него можно избавиться. Поэтому из векторизации, по-видимому, выжат максимум.

<h3>Вынесем векторную реализацию в отдельную функцию:</h3>

In [16]:
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances, distance_metrics
import copy
import math


def fit_kmeans(X, k=2, metric='euclidean', max_iter=1000, random_state=322):
        """
        Процедура обучения k-means
        """
        
        # Инициализация генератора случайных чисел
        np.random.seed(random_state)
        
        # Массив с метками кластеров для каждого объекта из X
        labels = np.empty(X.shape[0], dtype=np.int64)
        new_labels = np.empty(X.shape[0], dtype=np.int64)
        
        # Массив с центроидами кластеров, заполняем случайными числами в соответствии с диапазоном входных данных
        centroids = np.empty((k, X.shape[1]), dtype=np.float64)
        new_centroids = np.empty((k, X.shape[1]), dtype=np.float64)
        for i in xrange(X.shape[1]):
            min = X[:,i].min()
            max = X[:,i].max()
            centroids[:, i] = min + np.random.random(k)*(max-min)
            
                
        for it in xrange(max_iter):
            #заполняем new_labels - ближайший центроид для каждого объекта
            new_labels = distance_metrics()[metric](centroids, X).argmin(axis=0)
                        
            #считаем новые координаты центроидов
            for cen in xrange(k):
                new_centroids[cen] = np.average(X[new_labels == cen], axis=0)
            
            #если нет ни одной точки в кластере, снова случайно выставляем этот центроид
            empty_clusters = np.argwhere(np.isnan(new_centroids).any(axis=1))
            new_centroids[empty_clusters] = centroids[empty_clusters]         
            
            #если среднее смещение центроида меньше eps, то заканчиваем
            #delta = float(np.trace(distance_metrics()[self.metric](self.centroids, new_centroids))/self.k)
            #eps = 0.001
            #if delta < eps:
            
            #если ни один объект не поменял свой кластер, то заканчиваем
            if np.array_equal(new_labels, labels):
                np.copyto(centroids, new_centroids)
                np.copyto(labels, new_labels)
                #print 'iterations = ', it
                break
            
            np.copyto(centroids, new_centroids)
            np.copyto(labels, new_labels)

        return labels

Реализация через функцию не отличается по времени от реализации через класс:

In [17]:
X = np.array(df_norm)
%timeit -n 3 -r 3 fit_kmeans(X, k=9, random_state=322)

3 loops, best of 3: 560 ms per loop


Сама кластеризация такая же:

In [18]:
labels = fit_kmeans(X, k=9, random_state=322)
print 'Adjusted rand score = ', adjusted_rand_score(true_labels, labels)

Adjusted rand score =  1.0


In [19]:
X = np.array(df_norm)
profile_print(fit_kmeans, X, 9)

Timer unit: 1e-06 s

Total time: 0.586731 s
File: <ipython-input-16-10d23a538871>
Function: fit_kmeans at line 7

Line #      Hits         Time  Per Hit   % Time  Line Contents
     7                                           def fit_kmeans(X, k=2, metric='euclidean', max_iter=1000, random_state=322):
     8                                                   """
     9                                                   Процедура обучения k-means
    10                                                   """
    11                                                   
    12                                                   # Инициализация генератора случайных чисел
    13         1           18     18.0      0.0          np.random.seed(random_state)
    14                                                   
    15                                                   # Массив с метками кластеров для каждого объекта из X
    16         1           20     20.0      0.0          labels = np.empty(X.s

<h2>Numba</h2>

Реализация через класс:

In [20]:
from numba import jit, jitclass
from numba import int8, int64, float64
from sklearn.base import ClusterMixin, BaseEstimator
from numpy.linalg import norm
from sklearn.metrics.pairwise import distance_metrics, euclidean_distances
import copy
import cmath


spec = [
    ('k', int64),
    ('max_iter', int64),
    ('labels', int64[:]),
    ('centroids', float64[:,:]),
    #('metric', int8[:]),
    ('random_state', int64),
]

@jitclass(spec)
class Kmeans_numba(object): 
    
    def __init__(self, k=2, max_iter=1000, random_state=322):
        """
        Инициализация метода
        :k - количество кластеров
        :metric - функция расстояния между объектами
        :max_iter - максиальное количество итераций
        :random_state - seed для инициализации генератора случайных чисел
        """
        
        self.k = k
        self.random_state = random_state
        #self.metric = metric
        self.max_iter = max_iter

    def fit(self, X):
        """
        Процедура обучения k-means
        """
        #try:
        #    X = np.array(X)
        #except:
        #    raise TypeError('Need pd.DataFrame or np.array')
        
        # Инициализация генератора случайных чисел
        np.random.seed(self.random_state)
        
        # Массив с метками кластеров для каждого объекта из X
        self.labels = np.empty(X.shape[0], dtype=np.int64)
        new_labels = np.empty(X.shape[0], dtype=np.int64)
        
        # Массив с центроидами кластеров, заполняем случайными числами в соответствии с диапазоном входных данных
        self.centroids = np.empty((self.k, X.shape[1]), dtype=np.float64)
        
        for i in xrange(X.shape[1]):
            min = X[:,i].min()
            max = X[:,i].max()
            self.centroids[:, i] = min + np.random.random(self.k)*(max-min)
        
        distance_matrix = np.empty((X.shape[0], self.k), dtype=np.float64)
                
        for it in xrange(self.max_iter):
            #print it
            #заполняем new_labels - ближайший центроид для каждого объекта
            for obj in xrange(X.shape[0]):
                min_dist = 1000.00
                for cen in xrange(self.k):
                    d = 0
                    for i in xrange(X.shape[1]):
                        d += (X[obj, i] - self.centroids[cen, i])**2
                    d = abs(cmath.sqrt(abs(d)))
                    if d < min_dist:
                        min_dist = d
                        new_labels[obj] = cen
                        
            #считаем новые координаты центроидов
            num_pts_in_cluster = np.zeros(self.k, dtype=np.int64)
            new_centroids = np.zeros((self.k, X.shape[1]), dtype=np.float64)
            for obj in xrange(X.shape[0]):
                new_centroids[new_labels[obj]] += X[obj]
                num_pts_in_cluster[new_labels[obj]] += 1
            for cen in xrange(self.k):
                if num_pts_in_cluster[cen] != 0:
                    new_centroids[cen] /= num_pts_in_cluster[cen]
                else:
                    new_centroids[cen] = self.centroids[cen]        
            
            #копируем, и, если ни один объект не поменял свой кластер, то заканчиваем
            flag = True
            for cen in xrange(self.k):
                self.centroids[cen] = new_centroids[cen]
            for obj in xrange(X.shape[0]):
                if self.labels[obj] != new_labels[obj]:
                    flag = False
                self.labels[obj] = new_labels[obj]
            
            if(flag):
                #print 'iter = ', it
                break

        return self

    def predict(self, X, y=None):
        """
        Процедура предсказания кластера
        
        Возвращает метку ближайшего кластера для каждого объекта
        """
        try:
            X = np.array(X)
        except:
            raise TypeError('Need pd.DataFrame or np.array')
        
        return distance_metrics()[self.metric](self.centroids, X).argmin(axis=0)

Стало медленнее почти в 2 раза, а само разбиение на кластеры такое же:

In [21]:
X = np.array(df_norm)
%timeit -n 3 -r 3 Kmeans_numba(k=9, random_state=322, max_iter=1000).fit(X)

3 loops, best of 3: 965 ms per loop


In [22]:
labels = Kmeans_numba(k=9, random_state=322, max_iter=1000).fit(X).labels
print 'Adjusted rand score = ', adjusted_rand_score(true_labels, labels)

Adjusted rand score =  1.0


<h4>Вынесем вычисления в отдельную функцию и используем numba:</h4>

In [23]:
from numba import jit
from numba import int8, int64, float64
from sklearn.metrics.pairwise import distance_metrics
import copy
import cmath
from cmath import sqrt

@jit(nopython=True)
def fit_kmeans_numba(X, k=2, max_iter=1000, metric='euclidean', random_state=322):
        """
        Процедура обучения k-means
        """
        # Инициализация генератора случайных чисел
        np.random.seed(random_state)
        
        # Массив с метками кластеров для каждого объекта из X
        labels = np.empty(X.shape[0], dtype=np.int64)
        new_labels = np.empty(X.shape[0], dtype=np.int64)
        
        # Массив с центроидами кластеров, заполняем случайными числами в соответствии с диапазоном входных данных
        centroids = np.empty((k, X.shape[1]), dtype=np.float64)
        
        for i in xrange(X.shape[1]):
            min = X[:,i].min()
            max = X[:,i].max()
            centroids[:, i] = min + np.random.random(k)*(max-min)
        
        distance_matrix = np.empty((X.shape[0], k), dtype=np.float64)
                
        for it in xrange(max_iter):
            #print it 
            #заполняем new_labels - ближайший центроид для каждого объекта
            for obj in xrange(X.shape[0]):
                min_dist = 1000.00
                for cen in xrange(k):
                    d = 0
                    for i in xrange(X.shape[1]):
                        d += (X[obj, i] - centroids[cen, i])**2
                    d = abs(cmath.sqrt(d))
                    if d < min_dist:
                        min_dist = d
                        new_labels[obj] = cen

                        
            #считаем новые координаты центроидов
            num_pts_in_cluster = np.zeros(k, dtype=np.int64)
            new_centroids = np.zeros((k, X.shape[1]), dtype=np.float64)
            for obj in xrange(X.shape[0]):
                new_centroids[new_labels[obj]] += X[obj]
                num_pts_in_cluster[new_labels[obj]] += 1
            for cen in xrange(k):
                if num_pts_in_cluster[cen] != 0:
                    new_centroids[cen] /= num_pts_in_cluster[cen]
                else:
                    new_centroids[cen] = centroids[cen]
            
            #копируем, и, если ни один объект не поменял свой кластер, то заканчиваем
            flag = True
            for cen in xrange(k):
                centroids[cen] = new_centroids[cen]
            for obj in xrange(X.shape[0]):
                if labels[obj] != new_labels[obj]:
                    flag = False
                labels[obj] = new_labels[obj]
            
            if(flag):
                #print 'iter = ', it
                break

        return labels

И тоже нет отличия по времени от класса:

In [24]:
%timeit -n 3 -r 3 fit_kmeans_numba(X, k=9, random_state=322)

3 loops, best of 3: 968 ms per loop


Numba замедляет почти в 2 раза. Алгоритм достаточно оптимальный, потому что по максимуму сохранены быстрые векторные операции. Хотя пришлось убрать многие вектроные функции, потому что numba их не поддерживает. Само разбиение снова такое же:

In [25]:
labels = fit_kmeans_numba(X, k=9, random_state=322)
print 'Adjusted rand score = ', adjusted_rand_score(true_labels, labels)

Adjusted rand score =  1.0


<h2>Cython</h2>

Добавим в быструю векторную реализацию типы данных и декораторы, а также определение класса через cdef:

In [26]:
%%cython -a
import numpy as np
cimport numpy as np
import copy
from sklearn.metrics.pairwise import distance_metrics
import cython

cdef class Kmeans(object): 
    
    cdef int k, max_iter, random_state
    cdef str metric
    #cdef np.ndarray[np.int64_t, ndim=1] labels
    #cdef np.ndarray[np.float64_t, ndim=2] centroids
    
    def __init__(self, int k=2, str metric='euclidean', int max_iter=1000, int random_state=0):
        """
        Инициализация метода
        :k - количество кластеров
        :metric - функция расстояния между объектами
        :max_iter - максиальное количество итераций
        :random_state - seed для инициализации генератора случайных чисел
        """
        
        self.k = k
        self.random_state = random_state
        self.metric = metric
        self.max_iter = max_iter

    @cython.boundscheck(False)
    @cython.cdivision(True)
    cpdef fit(self, np.ndarray[np.float64_t, ndim=2] X):
        """
        Процедура обучения k-means
        """            
        cdef np.ndarray[np.int64_t, ndim=1] new_labels, labels
        cdef np.ndarray[np.float64_t, ndim=2] new_centroids, centroids
        
        # Инициализация генератора случайных чисел
        np.random.seed(self.random_state)
        
        # Массив с метками кластеров для каждого объекта из X
        labels = np.empty(X.shape[0], dtype=np.int64)
        new_labels = np.empty(X.shape[0], dtype=np.int64)
        
        # Массив с центроидами кластеров, заполняем случайными числами в соответствии с диапазоном входных данных
        centroids = np.empty((self.k, X.shape[1]), dtype=np.float64)
        new_centroids = np.empty((self.k, X.shape[1]), dtype=np.float64)
        for i in xrange(X.shape[1]):
            min = X[:,i].min()
            max = X[:,i].max()
            centroids[:, i] = min + np.random.random(self.k)*(max-min)
  

        for it in xrange(self.max_iter):
            
            #заполняем new_labels - ближайший центроид для каждого объекта
            new_labels = distance_metrics()[self.metric](centroids, X).argmin(axis=0)
                        
            #считаем новые координаты центроидов
            for cen in xrange(self.k):
                new_centroids[cen] = np.average(X[new_labels == cen], axis=0)
            
            #если нет ни одной точки в кластере, снова случайно выставляем этот центроид
            empty_clusters = np.argwhere(np.isnan(new_centroids).any(axis=1))
            new_centroids[empty_clusters] = centroids[empty_clusters]         
            
            #если ни один объект не поменял свой кластер, то заканчиваем
            if np.array_equal(new_labels, labels):
                np.copyto(centroids, new_centroids)
                np.copyto(labels, new_labels)
                #print 'iterations = ', it
                break
            
            #self.centroids = copy.deepcopy(new_centroids)
            #self.labels = copy.deepcopy(new_labels)
            np.copyto(centroids, new_centroids)
            np.copyto(labels, new_labels)

        return labels

По сравнению с numpy время не поменялось, разбиение тоже:

In [27]:
%timeit -n 3 -r 3 Kmeans(k=9, random_state=322).fit(X)

3 loops, best of 3: 574 ms per loop


In [28]:
labels = Kmeans(k=9, random_state=322).fit(X)
print 'Adjusted rand score = ', adjusted_rand_score(true_labels, labels)

Adjusted rand score =  1.0


<h3>Вынесем алгоритм в отдельную функцию:</h3>

Реализация через циклы. Попробуем минимизировать количество желтых строк в основном цикле: добаавляем типы, убираем обращения к сторонним модулям.

In [50]:
%%cython -a
# cython: profile=True
# cython: linetrace=True

import numpy as np
cimport numpy as np
from sklearn.metrics.pairwise import pairwise_distances, distance_metrics
import copy
import math
from math import sqrt
import cython
from datetime import datetime

@cython.boundscheck(False)
@cython.cdivision(True)
cpdef fit_kmeans_cython(np.ndarray[double, ndim=2] X, long k=2, str metric='euclidean', long max_iter=1000, long random_state=322):
        """
        Процедура обучения k-means
        """ 
        print 'Start 0', datetime.now().time()
        cdef float min_dist, d, min, max
        
        
        # Инициализация генератора случайных чисел
        np.random.seed(random_state)
        
        # Массив с метками кластеров для каждого объекта из X 
        cdef long N_features, N_samples
        N_samples = X.shape[0]
        N_features = X.shape[1]
        
        cdef np.ndarray[long, ndim=1] labels, new_labels, num_pts_in_cluster
        labels = np.empty(N_samples, dtype=long)
        new_labels = np.empty(N_samples, dtype=long)
        num_pts_in_cluster = np.zeros(k, dtype=long)
        
        # Массив с центроидами кластеров, заполняем случайными числами в соответствии с диапазоном входных данных
        cdef np.ndarray[double, ndim=2] centroids, new_centroids, dist_matr
        centroids = np.empty((k, N_features), dtype=np.float64)
        
        for i in xrange(N_features):
            min = X[:,i].min()
            max = X[:,i].max()       
            centroids[:, i] = min + np.random.random(k)*(max-min)
                
        print 'Start 1', datetime.now().time()
                
        for it in xrange(max_iter):
            #print it
            
            new_centroids = np.zeros((k, N_features), dtype=float)
            #num_pts_in_cluster = np.zeros(k, dtype=np.int64)
            
            if it == 0:
                print 'it = ', it, datetime.now().time()
            
            #заполняем new_labels - ближайший центроид для каждого объекта          
            for obj in xrange(N_samples):               
                min_dist = 1000
                for cen in xrange(k):
                    #num_pts_in_cluster[cen] = 0
                    d = 0
                    #d = (X[obj, :] - centroids[cen, :]).dot((X[obj, :] - centroids[cen, :]).T)
                    for j in xrange(N_features):
                        #d += (X[obj, j] - centroids[cen, j])**2
                        d += 1
                    #print d
                    d = sqrt(abs(d))
                    if d < min_dist:
                        min_dist = d
                        new_labels[obj] = cen
                
                #считаем новые координаты центроидов
                new_centroids[new_labels[obj]] += X[obj]
                num_pts_in_cluster[new_labels[obj]] += 1
            
            if it == 0:
                print 'it = ', it, datetime.now().time()
            
            #делим сумму на количество точек в кластере
            for cen in xrange(k):
                if num_pts_in_cluster[cen] != 0:
                    new_centroids[cen] /= num_pts_in_cluster[cen]
                else:
                    new_centroids[cen] = centroids[cen]        
                num_pts_in_cluster[cen] = 0
                
            
            #если ни один объект не поменял свой кластер, то заканчиваем
            flag = True
            for cen in xrange(k):
                centroids[cen] = new_centroids[cen]
                #new_centroids[cen] = np.zeros(N_features)
            for obj in xrange(N_samples):
                if labels[obj] != new_labels[obj]:
                    flag = False
                labels[obj] = new_labels[obj]
            
            if(flag):
                #print 'iter = ', it
                break            
        return labels

Стало медленне в 6 раз, по сравнению с Numpy, хотя в основном цикле почти не осталось желтых строк. Осталось только обнуление массива np.zeros, но оно работет быстрее, чем поэлементное обнуление:

In [21]:
%timeit -n 3 -r 3 fit_kmeans_cython(X, k=9, random_state=322)

3 loops, best of 3: 4.1 s per loop


Разбиение такое же:

In [51]:
labels = fit_kmeans_cython(X, k=9, random_state=322)
print 'Adjusted rand score = ', adjusted_rand_score(true_labels, labels)

Start 0 00:28:56.301767
Start 1 00:28:56.310659
it =  0 00:28:56.311770
it =  0 00:28:56.491364
Adjusted rand score =  -3.27156710521e-16


Профилирование показывает, что нет траты времени на обращение к сторонним модулям, то есть в этом направлении алгоритм уже не ускорить:

In [32]:
import pstats, cProfile 
cProfile.runctx("fit_kmeans_cython(X, k=9, random_state=322)", globals(), locals(), "Profile.prof") 

s = pstats.Stats("Profile.prof") 
s.strip_dirs().sort_stats("time").print_stats()

Sun Apr  2 16:38:18 2017    Profile.prof

         149 function calls in 3.718 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    3.715    3.715    3.718    3.718 _cython_magic_de98866bdb16a00bef0839796a3b1e4e.pyx:14(fit_kmeans_cython)
       72    0.003    0.000    0.003    0.000 {method 'reduce' of 'numpy.ufunc' objects}
       36    0.000    0.000    0.002    0.000 _methods.py:28(_amin)
       36    0.000    0.000    0.001    0.000 _methods.py:25(_amax)
        1    0.000    0.000    3.718    3.718 _cython_magic_de98866bdb16a00bef0839796a3b1e4e.pyx:14(fit_kmeans_cython (wrapper))
        1    0.000    0.000    3.718    3.718 <string>:1(<module>)
        1    0.000    0.000    3.718    3.718 {_cython_magic_de98866bdb16a00bef0839796a3b1e4e.fit_kmeans_cython}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




<pstats.Stats instance at 0x7eff736bac20>

<h3>Векторная реализация на cython:</h3>

Попоробуем ускорить быструю векторную реализацию - добавим к ней типы данных и декораторы:

In [33]:
%%cython -a
import numpy as np
cimport numpy as np
from sklearn.metrics.pairwise import pairwise_distances, distance_metrics
import copy
import math
import cython
cimport cython


@cython.boundscheck(False)
@cython.cdivision(True)
cpdef fit_kmeans_cython(np.ndarray[np.float64_t, ndim=2] X, long k=2, str metric='euclidean', long max_iter=1000, long random_state=322):
        """
        Процедура обучения k-means
        """
        
        # Инициализация генератора случайных чисел
        np.random.seed(random_state)
        
        cdef np.float64_t min, max
        
        # Массив с метками кластеров для каждого объекта из X
        cdef np.ndarray[long, ndim=1] labels, new_labels
        labels = np.empty(X.shape[0], dtype=np.int64)
        new_labels = np.empty(X.shape[0], dtype=np.int64)
        
        # Массив с центроидами кластеров, заполняем случайными числами в соответствии с диапазоном входных данных
        cdef np.ndarray[np.float64_t, ndim=2] centroids, new_centroids
        centroids = np.empty((k, X.shape[1]), dtype=np.float64)
        new_centroids = np.empty((k, X.shape[1]), dtype=np.float64)
        for i in xrange(X.shape[1]):
            min = X[:,i].min()
            max = X[:,i].max()
            centroids[:, i] = min + np.random.random(k)*(max-min)
        
        ## Your Code Here
                
        for it in xrange(max_iter):
            #заполняем new_labels - ближайший центроид для каждого объекта
            new_labels = distance_metrics()[metric](centroids, X).argmin(axis=0)
                        
            #считаем новые координаты центроидов
            for cen in xrange(k):
                new_centroids[cen] = np.average(X[new_labels == cen], axis=0)
            
            #если нет ни одной точки в кластере, снова случайно выставляем этот центроид
            empty_clusters = np.argwhere(np.isnan(new_centroids).any(axis=1))
            new_centroids[empty_clusters] = centroids[empty_clusters]         
            
            #если среднее смещение центроида меньше eps, то заканчиваем
            #delta = float(np.trace(distance_metrics()[self.metric](self.centroids, new_centroids))/self.k)
            #eps = 0.001
            #if delta < eps:
            
            #если ни один объект не поменял свой кластер, то заканчиваем
            if np.array_equal(new_labels, labels):
                np.copyto(centroids, new_centroids)
                np.copyto(labels, new_labels)
                #print 'iterations = ', it
                break
            
            np.copyto(centroids, new_centroids)
            np.copyto(labels, new_labels)

        return labels

Практически не ускоряет по сравнению с аналогичной векторной реализацией без cython:

In [34]:
%timeit -n 3 -r 3 fit_kmeans_cython(X, k=9, random_state=322)

3 loops, best of 3: 590 ms per loop


In [35]:
labels = fit_kmeans_cython(X, k=9, random_state=322)
print 'Adjusted rand score = ', adjusted_rand_score(true_labels, labels)

Adjusted rand score =  1.0


In [36]:
import pstats, cProfile 
cProfile.runctx("fit_kmeans_cython(X, k=9, random_state=322)", globals(), locals(), "Profile.prof") 

s = pstats.Stats("Profile.prof") 
s.strip_dirs().sort_stats("time").print_stats()

Sun Apr  2 16:38:57 2017    Profile.prof

         7902 function calls in 0.592 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.252    0.252    0.592    0.592 {_cython_magic_60dd4242b439e9208e0bd0569898b790.fit_kmeans_cython}
      501    0.101    0.000    0.101    0.000 {method 'reduce' of 'numpy.ufunc' objects}
       33    0.083    0.003    0.083    0.003 {numpy.core.multiarray.dot}
       33    0.077    0.002    0.248    0.008 pairwise.py:162(euclidean_distances)
       66    0.050    0.001    0.050    0.001 {numpy.core.multiarray.einsum}
      297    0.008    0.000    0.079    0.000 _methods.py:53(_mean)
      297    0.003    0.000    0.085    0.000 function_base.py:857(average)
       33    0.002    0.000    0.003    0.000 numeric.py:2476(array_equal)
     1419    0.002    0.000    0.002    0.000 {isinstance}
      297    0.002    0.000    0.002    0.000 _methods.py:43(_count_reduce_items)
       66    

<pstats.Stats instance at 0x7eff736ba290>

Для сравнения время работы встроенного в sklearn алгорима:

In [37]:
from sklearn.cluster import KMeans
%timeit -n 3 -r 3 KMeans(n_clusters=9, random_state=322).fit(X)

3 loops, best of 3: 4.04 s per loop


<b>Итог:</b> быстрее всего работает векторная реализация с numpy. Numba и cython не дают такого ускорения.

In [38]:
df = pd.read_csv('speed_results.csv')
df

,Алгоритм,Время
0,Через циклы,1.5 мин
1,Вектроный класс,551 мс
2,Векторная функция,560 мс
3,Numba класс,965 мс
4,Numba функция,968 мс
5,Cython класс,574 мс
6,Cython функция через циклы,3.75 сек
7,Cython векторная функция,590 мс
8,sklearn.KMeans,4.04 сек
